# Neural Networks

 We'll build a neural network from scratch, specifically focusing on how it makes predictions using forward propagation.

 Deep Learning libraries have the entire training and prediction processes implemented, so in practice, you wouldn't need to build a neural network from scratch. This exercise just gives you a clear understanding of how neural networks function under the hood.

<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width="600px">

In [1]:
# Import Libraries

import numpy as np

In [3]:
# Initialize weights and biases

weights = np.around(np.random.uniform(size=6), decimals=2) # initialize the weights
biases = np.around(np.random.uniform(size=3), decimals=2)  # initialize the biases

print(weights)
print(biases)

[0.9  0.42 0.46 0.92 0.66 0.05]
[0.06 0.34 0.42]


Compute the output for a given input, $x_1$ and $x_2$.

In [4]:
# Give the inputs, x1 and x2.

x_1 = 0.5 # input 1
x_2 = 0.85 # input 2

print('x1 is {} and x2 is {}'.format(x_1, x_2))

x1 is 0.5 and x2 is 0.85


In [7]:
# Hidden Layer

z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]
print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]
print('The weighted sum of the inputs at the Second node in the hidden layer is {}'.format(z_12))

The weighted sum of the inputs at the first node in the hidden layer is 0.867
The weighted sum of the inputs at the Second node in the hidden layer is 1.352


In [10]:
# Compute the activation using a sigmoid activation function

a_11 = 1.0 / (1.0 + np.exp(-z_11))
print('The activation of the first node in the hidden layer is {}'.format(np.around(a_11, decimals=4)))

a_12 = 1.0 / (1.0 + np.exp(-z_12))
print('The activation of the first node in the hidden layer is {}'.format(np.around(a_12, decimals=4)))

The activation of the first node in the hidden layer is 0.7041
The activation of the first node in the hidden layer is 0.7945


In [12]:
# Compute the weighted sum of these inputs to the node in the output layer.

z_2 = a_11 * weights[4] + a_12 * weights[5] + biases[2]
print('The weighted sum of the inputs at the node in the output layer is {}'.format(np.around(z_2, decimals=4)))

The weighted sum of the inputs at the node in the output layer is 0.9244


In [13]:
# Compute the output of the network as the activation of the node in the output layer.

a_2 = 1.0 / (1.0 + np.exp(-z_2))
print('The output of the network for x1 = 0.5 and x2 = 0.85 is {}'.format(np.around(a_2, decimals=4)))

The output of the network for x1 = 0.5 and x2 = 0.85 is 0.7159


## Build a Neural Network

In [14]:
# Define the structure of the network.

n = 2 # number of inputs
num_hidden_layers = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer

In [16]:
# Function to inititailize the weights and the biases in the network to random numbers.
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):

    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}

    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):

        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer]

        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }

        num_nodes_previous = num_nodes

    return network # return the network

In [17]:
# Create a small network that:
# takes 5 inputs
# has three hidden layers
# has 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer
# has 1 node in the output layer

small_network = initialize_network(5, 3, [3, 2, 3], 1)

In [18]:
# Function to compute Weighted Sum at Each Node

def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

In [19]:
# Generate 5 inputs that we can feed to small_network.

from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


In [20]:
# Compute the weighted sum at the first node in the first hidden layer

node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']

weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias)
print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))

The weighted sum at the first node in the hidden layer is 1.6405


In [21]:
# A Function to compute Node Activation

def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

In [22]:
node_output  = node_activation(compute_weighted_sum(inputs, node_weights, node_bias))
print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))


The output of the first node in the hidden layer is 0.8376


### Forward Propagation

1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
6. Repeat steps 2 - 5 until we compute the output of the output layer.

In [23]:
def forward_propagate(network, inputs):

    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer

    for layer in network:

        layer_data = network[layer]

        layer_outputs = []
        for layer_node in layer_data:

            node_data = layer_data[layer_node]

            # compute the weighted sum and the output of each node at the same time
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))

        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))

        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

In [24]:
predictions = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))

The outputs of the nodes in hidden layer number 1 is [np.float64(0.8376), np.float64(0.8603), np.float64(0.8615)]
The outputs of the nodes in hidden layer number 2 is [np.float64(0.8766), np.float64(0.855)]
The outputs of the nodes in hidden layer number 3 is [np.float64(0.7275), np.float64(0.665), np.float64(0.8073)]
The predicted value by the network for the given input is 0.7811
